This is another attempt using tutorial from https://towardsdatascience.com/variational-autoencoder-demystified-with-pytorch-implementation-3a06bee395ed

In [ ]:
import numpy as np
import torch
from scipy.stats import norm

# Testing $D_{KL}$ from torch

Theoretical value for kl divergence between two univariate normal distribution

In [22]:
def kl_theory(mean1,sd1,mean2,sd2):
    return np.log(sd2/sd1)+(sd1*sd1+(mean1-mean2)*(mean1-mean2))/(2*sd2*sd2)-0.5

Testing the function with random values where $\mu \in (-100,100)$ and $\sigma \in (0,100)$

In [115]:
#this function seems ok
sq_diff = 0
for k in range(1000):
    (mu1, mu2, sd1, sd2)= (np.random.random_sample()*200-100, 
                           np.random.random_sample()*200-100,
                           np.random.random_sample()*100,
                           np.random.random_sample()*100)
    test_val = torch.distributions.kl.kl_divergence(
        torch.distributions.Normal(mu1,sd1),
        torch.distributions.Normal(mu2,sd2))
    theory_val = kl_theory(mu1,sd1,mu2,sd2)
    sq_diff = sq_diff + (theory_val-test_val)*(theory_val-test_val)
    
mean_sq_diff = sq_diff/1000
print('Mean Squared Difference = %1.5f' %mean_sq_diff)

Mean Squared Difference = 0.00057


## KL divergence from the website 

(probably not gonna use this

In [ ]:
def kl_divergence(self, z, mu, std):
        # --------------------------
        # Monte carlo KL divergence
        # --------------------------
        # 1. define the first two probabilities (in this case Normal for both)
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        # 2. get the probabilities from the equation
        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        # kl
        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)
        return kl

# Tensor learning/investigation

Turning data from truth into tensor that we can work with

In [110]:
np.random.seed(508)
dat = np.random.normal(0,5,10)

In [111]:
sample = torch.tensor(dat)
print(sample)
print(torch.add(4*sample,torch.ones_like(sample)))

tensor([  3.6381,  -1.6319,  -6.1138,  -1.6868,   1.8680,  -1.2100, -10.4189,
          7.6481,   3.7290,   4.0272], dtype=torch.float64)
tensor([ 15.5522,  -5.5275, -23.4553,  -5.7472,   8.4719,  -3.8400, -40.6756,
         31.5922,  15.9160,  17.1087], dtype=torch.float64)


# Experiments with $q(z|x)$

In [146]:
torch.manual_seed(508)

In [147]:
p=torch.distributions.Normal(0,1)

In [148]:
qzx=torch.distributions.Normal(loc=torch.add(4*sample,torch.ones_like(sample)),
                               scale=torch.ones_like(sample))

In [149]:
z = qzx.rsample()

Sampling from $q$ will generate a sample for each parameter

In [150]:
print(z)

tensor([ 12.8077,  -5.7186, -22.4107,  -7.1926,   9.2840,  -1.8070, -39.1750,
         31.7305,  16.7579,  17.5327], dtype=torch.float64)


We can sum in a tensor

In [151]:
print(torch.distributions.kl.kl_divergence(qzx,p).sum(-1))
print(q.log_prob(z))
print(q.log_prob(z).sum(-1))

tensor(2070.3623, dtype=torch.float64)
tensor([  -43.0605,    -4.5853,    -7.9904, -1022.7177,    -2.6962,  -654.3091,
         -146.5263,  -634.6454,    -1.5597,   -41.3007], dtype=torch.float64)
tensor(-2559.3913, dtype=torch.float64)


We can probably work out $D_{KL}$ this way, but <b>does it make sense to sum over the divergences</b>?

## Working out expectations

The examples from the tutorial

They first define the likelihood function where the mean is parametrized by <code>x_hat</code> (apparently a sample)

In [ ]:
def gaussian_likelihood(self, x_hat, logscale, x):
        scale = torch.exp(logscale)
        mean = x_hat
        dist = torch.distributions.Normal(mean, scale)

        # measure prob of seeing image under p(x|z)
        log_pxz = dist.log_prob(x)
        return log_pxz.sum(dim=(1, 2, 3))

Then they simply define $\mathbb{E}_{q(z|x)}\log p(x|z)$ (called recon_loss here) using <code>gaussian_likelihood</code>

In [ ]:
recon_loss = self.gaussian_likelihood(x_hat, self.log_scale, x)

Several problems:
- gaussian_likelihood simply sum over the log probabilities of $p(x|z)$, how does that calculate the expectation when the whole thing is given by $\int q(z|x)\log p(x|z) dz$?
- the function only sum over finitely many points, how does that calculate the integration accurately?

Ideas:
<ol>
<li>there is a integration option using <code>torch.trapz</code>, but implementing might be difficult</li>
</ol>